# Common Variables Preparation for Machine Learning Models
* Author: 
    [Dr. Wenwen Zhang](http://wenwenz.com)
* Objective: 
    This notebook prepares common variables in 
    1. National AV public opinion survey
    2. seattle travel survey data
    3. 2017 NHTS data
    
Acknowledgement:
    
> The **national public opinion survey** conducted by [Dr. Robert Noland](https://bloustein.rutgers.edu/noland/) and Sicheng Wang at Rutgers University and [Dr. Andrew Mondschein](https://www.arch.virginia.edu/people/andrew-mondschein) and Zhiqiu Jiang at University of Virginia in 2017. 

> The **[2017 National Household Travel Survey (NHTS)](https://nhts.ornl.gov/)** is a national travel survey dataset conducted by Federal Highway Administration (FHWA). The data are publicly available. 

> The **seattle travel survey data** are collected and shared by [Puget Sound Regional Council](https://www.psrc.org/)

In [12]:
# import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
favfile  = "../SurveyData_latest.csv"
seattle_p_file = "../2017-pr1-R-2-person.csv" # seattle person data
seattle_hh_file = "../2017-pr1-R-1-household.csv" # Seattle household data
nhts_p_file = "../"
nhts_hh_file = "../"

## [1] Prepare National Opinion Survey Data

In [22]:
df = pd.read_csv(favfile)
common_v = ["q1gender",
            "hispanic_adj",
            "race_adj",
            "generation",
            "age", 
            "q10freq_rideshare",
            "q27commute_mode",
            'car_n',
            "q33edu",
            "q34_0hhsize",
            "q34children",
            "income_adj"
               ] 
target_v = ["purchase", # target variable for interest in purchasing AVs
            "use_taxiAV" # target variable for interest in using shared AVs
           ]

X = df[common_v + target_v]

# common explanatory variables recoding:
X["generation"][X["age"] <=34] = "Millennials"
X["generation"][(X["age"] >=35) & (X["age"] <=54) ] = "Generation X"
X["generation"][(X["age"] > 54) & (X["age"] <=74) ] = "Baby Boomers"
X["generation"][X["age"] >= 75] = "The Silent Generation"

X["age"][X["age"]>=85] = "85 or years older"
X["age"][(X["age"]<= 84) & (X["age"]>= 75)] = "75-84 years"
X["age"][(X["age"]<= 74) & (X["age"]>= 65)] = "65-74 years"
X["age"][(X["age"]<= 64) & (X["age"]>= 55)] = "55-64 years"
X["age"][(X["age"]<= 54) & (X["age"]>= 45)] = "45-54 years"
X["age"][(X["age"]<= 44) & (X["age"]>= 35)] = "35-44 years"
X["age"][(X["age"]<= 34) & (X["age"]>= 25)] = "25-34 years"
X["age"][X["age"]< 25] = "Under 25 years"

X["q27commute_mode"][X["q27commute_mode"]=='Neither work nor go to school'] = "None"
X["q27commute_mode"][X["q27commute_mode"]=='Work at home'] = "None"
X["q27commute_mode"][X["q27commute_mode"]=='Other method'] = "Other"

X["car_n"][X["car_n"]== 6] = "6 or more"

X["q33edu"][X["q33edu"]=='Prefer not to say'] = "None"
X["q33edu"][X["q33edu"] == 'Master<U+0092>s degree'] = "Master or above"
X["q33edu"][X["q33edu"] == 'Graduate or professional degree'] = "Master or above"

X["q34_0hhsize"][X["q34_0hhsize"] == 'Prefer not to say'] = "None"
X["q34children"][X["q34children"] == 'Prefer not to say'] = "None"
X["income_adj"][(X["income_adj"] == "$100,000 to under $150,000") | (X["income_adj"] == "$150,000 to under $200,000") | (X["income_adj"] == "$200,000 or greater")] = "$100,000 or greater"

# target variables recoding into binary variables
X["purchase"][X["purchase"]==-1] = 0 # -1 not certain, 0 not interested, 1 interested. 
X["use_taxiAV"][X["use_taxiAV"]==-1] = 0 # -1 not certain, 0 not interested, 1 interested. 

# replace nan as "None" and set all variables to be categorical
for col in list(X):
    if col not in target_v:
        X[col].replace(np.nan,'None',inplace=True)
    X[col] = X[col].astype("category")

# update column names
X.columns = ["gender", 
             "hispanic", 
             "race", 
             "generation",
             "age", 
             "freq_rideshare", 
             "commute_mode", 
             "car_n", 
             "edu", 
             "hhsize", 
             "kid_n", 
             "income",
             "Target_PAV",
             "Target_SAV"]


for v in list(X):
    print v
    print X[v].value_counts()
    print "-"*50

X = pd.get_dummies(X) #update categorical variables into dummies

for col in list(X):
    if "None" in col:
        X = X.drop(col,1) # drop Dummies for None Categories
X = X.drop('Target_SAV_0.0',1) # drop Dummies for Target_SAV zero
X = X.drop('Target_PAV_0.0',1) # drop Dummies for None Categories     


gender
Female               425
Male                 398
Prefer not to say      5
Other                  1
None                   1
Name: gender, dtype: int64
--------------------------------------------------
hispanic
No      673
Yes     144
None     13
Name: hispanic, dtype: int64
--------------------------------------------------
race
White               617
African American    117
Asian                44
Other                36
None                 16
Name: race, dtype: int64
--------------------------------------------------
generation
Generation X             288
Baby Boomers             260
Millennials              255
The Silent Generation     27
Name: generation, dtype: int64
--------------------------------------------------
age
45-54 years          151
25-34 years          148
55-64 years          138
35-44 years          137
65-74 years          122
Under 25 years       107
75-84 years           23
85 or years older      4
Name: age, dtype: int64
---------------------------

> It is noted that the dataset is imbalanced, as there are significnatly less "1" than "0" cases in both target variables.

## [2] Prepare Seattle Household Travel Survey Data

In [25]:
df_seattle_p = pd.read_csv(seattle_p_file)
df_seattle_p = df_seattle_p[["hhid","gender", "race_hisp",  "race_asian","race_afam",  "race_white", "race_noanswer", "age", "mode_freq_5", "commute_mode","education","av_interest_1", "av_interest_5"]]
df_seattle_hh = pd.read_csv(seattle_hh_file)
df_seattle_hh = df_seattle_hh[["hhid","vehicle_count", "hhsize", "numchildren", "hhincome_detailed", "hhincome_broad"]]

# common explanatory variables recoding:
df_seattle_p["gender"][df_seattle_p["gender"]==1] = 'Male'
df_seattle_p["gender"][df_seattle_p["gender"]==2] = 'Female'
df_seattle_p["gender"][df_seattle_p["gender"]==3] = 'Other'
df_seattle_p["gender"][df_seattle_p["gender"]==4] = 'Prefer not to say'

df_seattle_p["race_hisp"][df_seattle_p["race_hisp"]==1] = 'Yes'
df_seattle_p["race_hisp"][df_seattle_p["race_hisp"]==0] = 'No'

df_seattle_p["race"] = 'Other'
df_seattle_p["race"][df_seattle_p["race_afam"]==1] = 'African American'
df_seattle_p["race"][df_seattle_p["race_asian"]==1] = 'Asian'
df_seattle_p["race"][df_seattle_p["race_white"]==1] = 'White'
df_seattle_p["race"][df_seattle_p["race_noanswer"]==1] = 'None'
df_seattle_p = df_seattle_p.drop("race_afam",1)
df_seattle_p = df_seattle_p.drop("race_asian",1)
df_seattle_p = df_seattle_p.drop("race_white",1)
df_seattle_p = df_seattle_p.drop("race_noanswer",1)

df_seattle_p["generation"] = "None"
df_seattle_p["generation"][df_seattle_p["age"] <=6] = "Millennials"
df_seattle_p["generation"][(df_seattle_p["age"] > 6) & (df_seattle_p["age"] <=8) ] = "Generation X"
df_seattle_p["generation"][(df_seattle_p["age"] > 8) & (df_seattle_p["age"] <=10) ] = "Baby Boomers"
df_seattle_p["generation"][df_seattle_p["age"] >= 11] = "The Silent Generation"

df_seattle_p["age"][df_seattle_p["age"] <= 5 ]="Under 25 years"
df_seattle_p["age"][df_seattle_p["age"] == 6 ]="25-34 years"
df_seattle_p["age"][df_seattle_p["age"] == 7 ]="35-44 years"
df_seattle_p["age"][df_seattle_p["age"] == 8 ]="45-54 years"
df_seattle_p["age"][df_seattle_p["age"] == 9 ]="55-64 years"
df_seattle_p["age"][df_seattle_p["age"] == 10]= "65-74 years"
df_seattle_p["age"][df_seattle_p["age"] == 11]= "75-84 years"
df_seattle_p["age"][df_seattle_p["age"] == 12]= "85 or years older"

df_seattle_p["mode_freq_5"][df_seattle_p["mode_freq_5"] <= 3] = "Several times per week"
df_seattle_p["mode_freq_5"][df_seattle_p["mode_freq_5"] == 4 ] = "Once per week"
df_seattle_p["mode_freq_5"][df_seattle_p["mode_freq_5"] == 5 ] = "Once per month"
df_seattle_p["mode_freq_5"][df_seattle_p["mode_freq_5"] == 6 ] = "A few times per year"
df_seattle_p["mode_freq_5"][df_seattle_p["mode_freq_5"] == 7 ] = "Never"

df_seattle_p["commute_mode"][df_seattle_p["commute_mode"] <=3 ] = "Personal Automobiles"
df_seattle_p["commute_mode"][(df_seattle_p["commute_mode"] == 6) | (df_seattle_p["commute_mode"] == 7)] = "Biking & Walking"
df_seattle_p["commute_mode"][(df_seattle_p["commute_mode"] == 15) | (df_seattle_p["commute_mode"] == 16)] = "Taxi, Uber, or Lyft"
df_seattle_p["commute_mode"][(df_seattle_p["commute_mode"] >=10) & (df_seattle_p["commute_mode"] <=14)] = "Public Transit"
df_seattle_p["commute_mode"][df_seattle_p["commute_mode"] == 8 ] = "Public Transit"
df_seattle_p["commute_mode"][df_seattle_p["commute_mode"] <100 ] = "Other"

df_seattle_p["education"][df_seattle_p["education"] == 1 ] = "Less than high school"
df_seattle_p["education"][df_seattle_p["education"] == 2 ] = "High school or GED"
df_seattle_p["education"][df_seattle_p["education"] == 3 ] = "Some College"
df_seattle_p["education"][df_seattle_p["education"] == 4 ] = "Some College"
df_seattle_p["education"][df_seattle_p["education"] == 5 ] = "Two-year college degree"
df_seattle_p["education"][df_seattle_p["education"] == 6 ] = "Four-year college degree"
df_seattle_p["education"][df_seattle_p["education"] == 7 ] = "Master or above"


df_seattle_hh["vehicle_count"][df_seattle_hh["vehicle_count"] >=6 ] = "6 or more"
df_seattle_hh["hhsize"][df_seattle_hh["hhsize"] >=6 ] = "6 or more"
df_seattle_hh["numchildren"][df_seattle_hh["numchildren"] >=4 ] = "4 or more"

df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] <=2 ] = "Under $25,000"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==3 ] = "$25,000 to under $50,000"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==4 ] = "$25,000 to under $50,000"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==5 ] = "$50,000 to under $100,000"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==6 ] = "$50,000 to under $100,000"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==7 ] = "$100,000 or greater"#"$100,000 to under $150,000"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==8 ] = "$100,000 or greater"#"$150,000 to under $200,000"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==9 ] = "$100,000 or greater"#"$200,000 or greater"
df_seattle_hh["hhincome_detailed"][df_seattle_hh["hhincome_detailed"] ==10 ] = "$100,000 or greater"#"$200,000 or greater"
df_seattle_hh["hhincome_detailed"][(df_seattle_hh["hhincome_detailed"] ==11) & (df_seattle_hh["hhincome_broad"] ==1)] = "Under $25,000"
df_seattle_hh["hhincome_detailed"][(df_seattle_hh["hhincome_detailed"] ==11) & (df_seattle_hh["hhincome_broad"] ==2)] = "$25,000 to under $50,000"
df_seattle_hh["hhincome_detailed"][(df_seattle_hh["hhincome_detailed"] ==11) & (df_seattle_hh["hhincome_broad"] ==3)] = "$50,000 to under $100,000"
df_seattle_hh["hhincome_detailed"][(df_seattle_hh["hhincome_detailed"] ==11) & (df_seattle_hh["hhincome_broad"] ==4)] = "$50,000 to under $100,000"
df_seattle_hh["hhincome_detailed"][(df_seattle_hh["hhincome_detailed"] ==11) & (df_seattle_hh["hhincome_broad"] ==5)] = "$100,000 or greater"
df_seattle_hh["hhincome_detailed"][(df_seattle_hh["hhincome_detailed"] ==11) & (df_seattle_hh["hhincome_broad"] ==6)] = "None"
df_seattle_hh = df_seattle_hh.drop("hhincome_broad",1)

# target features recoding
df_seattle_p["av_interest_5"][df_seattle_p["av_interest_5"] <=2 ] = 1
df_seattle_p["av_interest_5"][df_seattle_p["av_interest_5"] > 2 ] = 0

df_seattle_p["av_interest_1"][df_seattle_p["av_interest_1"] <=2 ] = 1
df_seattle_p["av_interest_1"][df_seattle_p["av_interest_1"] > 2 ] = 0

# join hhousehold information to person table
df_seattle = df_seattle_p.merge(df_seattle_hh, how = "left", left_on = "hhid", right_on = "hhid")

# reset column names
df_seattle.columns = ["hhid", "gender", "hispanic", "age", "freq_rideshare", "commute_mode", "edu", "Target_PAV", "Target_SAV", "race", "generation", "car_n", "hhsize", "kid_n", "income"]

df_seattle = df_seattle.drop("hhid", 1)
df_seattle = df_seattle[["gender", "hispanic", "race", "generation","age", "freq_rideshare", "commute_mode", "car_n", "edu", "hhsize", "kid_n", "income", "Target_PAV", "Target_SAV"]]

#drop purchase as nan
for col in list(df_seattle):
    if "Target" not in col:
        df_seattle[col].replace(np.nan,'None',inplace=True)
    df_seattle[col] = df_seattle[col].astype("category")
    print col
    print df_seattle[col].value_counts()
    print "-"*50
    
df_seattle = pd.get_dummies(df_seattle)

for col in list(df_seattle):
    if "None" in col:
        df_seattle = df_seattle.drop(col,1)

gender
Female               3095
Male                 3020
Prefer not to say     119
Other                  20
Name: gender, dtype: int64
--------------------------------------------------
hispanic
No      5189
None     853
Yes      212
Name: hispanic, dtype: int64
--------------------------------------------------
race
White               3778
Other               1093
Asian                872
None                 403
African American     108
Name: race, dtype: int64
--------------------------------------------------
generation
Millennials              2944
Generation X             1843
Baby Boomers             1227
The Silent Generation     240
Name: generation, dtype: int64
--------------------------------------------------
age
25-34 years          1719
Under 25 years       1225
35-44 years          1151
45-54 years           692
55-64 years           676
65-74 years           551
75-84 years           202
85 or years older      38
Name: age, dtype: int64
----------------------------

## [3] Prepare 2017 NHTS dataset